In [ ]:
# awk 'BEGIN {printf("##fileformat=VCFv4.2\n##INFO=<ID=END,Number=1,Type=Integer>\n#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n");} {printf("%s\t%s\t.\t%s\t%s\t.\t.\t.\n",$1,int($2)+1,$5,$8);}' test.bed > test2.vcf

In [2]:
import pandas as pd
df=pd.read_csv('test.bed',sep='\t')
df.fillna('-',inplace=True)
df['start']=df['start'].astype(int)+1
# df.to_csv('check.txt',sep='\t')
df['ID3']=df['ref'].astype(str)+"/"+df['alt'].astype(str)
df['ID4']=1
df[['chr','start','stop','ID3','ID4']].to_csv('vep_input.txt',index=False,sep='\t',header=False)

# for hg19 --  upload check.txt to https://grch37.ensembl.org/Homo_sapiens/Tools/VEP?db=core;tl=nAhnFMX4JsxRgd0h-10490081

In [36]:
# intersect between vep files and our transcripts
import pandas as pd
ref=pd.read_csv('neo_transcript.txt',sep='\t')
df=pd.read_csv('refseq.txt',sep='\t') #,usecols=['Location','SYMBOL','Feature','HGVSc','HGVSp', 'Feature_original',])
df['Feature2']=df['Feature'].str.split('.').str[0]
df_merged=pd.merge(ref,df,how='inner',on=['SYMBOL','Feature2'])
df2=pd.read_csv('ensembl.txt',sep='\t')#,usecols=['Location','SYMBOL','Feature','HGVSc','HGVSp'])
df2['Feature2']=df2['Feature'].str.split('.').str[0]
df2_merged=pd.merge(ref,df2,how='inner',on=['SYMBOL','Feature2'])
# merge refseq and ensembl
df_concat=pd.concat([df_merged,df2_merged])
df_concat.to_csv('refseq_ensembl_combined.txt',sep='\t',index=False)




In [39]:
#manually copy over 4th column from check.txt to test.bed for now
ref=pd.read_csv('test.bed',sep='\t')
x =pd.read_csv('refseq_ensembl_combined.txt',sep='\t',usecols=['Location','SYMBOL','Feature','UPLOADED_ALLELE','HGVSc','HGVSp'])
x['chr']=x['Location'].str.split(':').str[0]
x['temp']=x['Location'].str.split(':').str[1]
x['start']=x['temp'].str.split('-').str[0].astype(int) - 1
x['start']=x['start'].astype(str)
x['stop']=x['temp'].str.split('-').str[1]

In [41]:
x['chr']=x['chr'].astype(str)

xx=pd.merge(ref,x,how='left',on=['chr','stop','UPLOADED_ALLELE'])
xx.drop_duplicates(inplace=True)
xx.to_csv('final.txt',sep='\t')